# Part 1 : Index Construction

## Data Crawling

In [ ]:
from googlesearch import search 
import tldextract  
import requests 
from bs4 import BeautifulSoup 
import csv   
from time import sleep 

# to search 
queries = ["Computer Science", "IITs in India", "Cities of Chattisgarh"]

#Loop for each query
for i in range(len(queries)):
    query = queries[i]
    # Loop for each search result
    count_files = 20
    urls = []
    for url in search(query,num=25, start= 1, stop=45, pause=2): 
        urls.append(url)
        
    for url in urls:
        print(url)
        try:
            respons = requests.get(url, timeout=5) #Response for the current URL
        except:
            continue
        sleep(0.5)
        soup = BeautifulSoup(respons.content, 'html5lib')
        paras=soup.find_all('p')  #Finding all the paragraph within the response
        para = "" #Initializing a empty paragraph
        
        count = 2 #To insure that file must contains 2 paragraph
        flag = 0
        
        for p in paras:   # Loop though each paragraph in response and append the contnet of that paragraph to  "para" so that we can save it as a continue paragraph
            if p!="":
                para += p.get_text()
                count-=1
            if (count==0):
                flag=1
                break
                
        if(flag):  #Perfom writing to file onlw when there is something in para
            ext_url = tldextract.extract(url)  #parsing the URL 
            domin_name = ext_url.domain   #Get the domain name from the parsed URL so that we can use it as a fine name

            with open(f'./d{21-count_files}_q{i+1}.txt', mode='wt', encoding='utf-8') as file:  #Wtite "para" to a txt file whose filename is a domain name of current URL
                file.write(para)
            count_files-=1
        
        if count_files==0:
            break

https://www.britannica.com/science/computer-science
https://www.edx.org/course/subject/computer-science
https://www.khanacademy.org/computing/computer-science
https://www.internationalstudent.com/study-computer-science/what-is-computer-science/
https://www.youtube.com/watch/Tzl0ELY_TiM
https://www.coursera.org/browse/computer-science
https://www.topuniversities.com/courses/computer-science-information-systems/guide
https://www.timeshighereducation.com/student/what-to-study/computer-science
https://www.computerscience.org/
https://arxiv.org/archive/cs
https://www.seas.harvard.edu/computer-science
https://engineering.buffalo.edu/computer-science-engineering.html
https://www.timeshighereducation.com/student/subjects/what-can-you-do-computer-science-degree
https://engineering.virginia.edu/departments/computer-science
https://www.codecademy.com/learn/paths/computer-science
https://www.ou.edu/coe/cs
https://www.usnews.com/best-graduate-schools/top-science-schools/computer-science-rankings
ht